In [ ]:
!pip install transformers evaluate datasets rouge_score -q

In [2]:
# Import essential libraries
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
from datasets import Dataset
from transformers import (
    TFAutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
import evaluate

In [43]:
# Configuration
num_workers = os.cpu_count()
epochs = 7
learning_rate = 2e-5

# Model 
model_name = 'vinai/bartpho-word'  

# Warnings configuration
warnings.filterwarnings('ignore')

# Load the model and tokenizer (TensorFlow version)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


All model checkpoint layers were used when initializing TFMBartForConditionalGeneration.

Some layers of TFMBartForConditionalGeneration were not initialized from the model checkpoint at vinai/bartpho-word and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
out_dir = '/kaggle/working'  # Output directory
train_path = '/kaggle/input/vietnews-dataset/train.csv'  # Path to training data
valid_path = '/kaggle/input/vietnews-dataset/valid.csv'  # Path to validation data
test_path = '/kaggle/input/vietnews-dataset/test.csv'  # Path to test data

# Convert datasets to Hugging Face format after cleaning
train_df["Content"] = train_df["Content"].astype(str)
valid_df["Content"] = valid_df["Content"].astype(str)
test_df["Content"] = test_df["Content"].astype(str)
train_df["Abstract"] = train_df["Abstract"].astype(str)
valid_df["Abstract"] = valid_df["Abstract"].astype(str)
test_df["Abstract"] = test_df["Abstract"].astype(str)

train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)

# Preprocess function to tokenize Vietnamese inputs and outputs
def preprocessing(examples):
    """
    Tokenizes the input Vietnamese text (Content) and prepares the model inputs for training.
    Uses the tokenizer's maximum length to handle longer documents.
    """
    inputs = [doc for doc in examples["Content"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    
    # Tokenizing Abstracts (targets)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Abstract"], max_length=1024, truncation=True, padding="max_length")
    
    # Adding labels to the inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenizing the Vietnamese datasets
tokenized_train = train_ds.map(preprocessing, batched=True)
tokenized_valid = valid_ds.map(preprocessing, batched=True)
tokenized_test = test_ds.map(preprocessing, batched=True)


Map:   0%|          | 0/105418 [00:00<?, ? examples/s]

Map:   0%|          | 0/22642 [00:00<?, ? examples/s]

Map:   0%|          | 0/22644 [00:00<?, ? examples/s]

In [45]:
tokenized_train

Dataset({
    features: ['Filename', 'Title', 'Abstract', 'Content', 'Keyword', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 105418
})

In [47]:
import tensorflow as tf

# Convert the Hugging Face dataset to a TensorFlow dataset
tf_train_dataset = tokenized_train.to_tf_dataset(
    columns=['input_ids', 'attention_mask'],  # Input features
    label_cols='labels',                      # Label column
    shuffle=True,
    batch_size=16,
    collate_fn=lambda x: {
        'input_ids': tf.ragged.constant([i['input_ids'] for i in x], dtype=tf.int32).to_tensor(),
        'attention_mask': tf.ragged.constant([i['attention_mask'] for i in x], dtype=tf.int32).to_tensor(),
        'labels': tf.constant([i['labels'] for i in x], dtype=tf.int32)
    }
)

# Verify a batch
for batch in tf_train_dataset.take(1):
    inputs, labels = batch
    print("Input IDs shape:", inputs['input_ids'].shape)
    print("Attention Mask shape:", inputs['attention_mask'].shape)
    print("Labels shape:", labels.shape)


Input IDs shape: (16, 1024)
Attention Mask shape: (16, 1024)
Labels shape: (16, 1024)


In [1]:
max_len = 512

# Giảm kích thước dữ liệu trong tập huấn luyện
tf_train_dataset = tf_train_dataset.map(lambda x, y: ({
    'input_ids': x['input_ids'][:, :max_len],  # Chỉ lấy 512 token đầu tiên
    'attention_mask': x['attention_mask'][:, :max_len]
}, y[:, :max_len]))  # Giảm nhãn về kích thước 512
model.fit(tf_train_dataset.batch(32), epochs=3)


NameError: name 'tf_train_dataset' is not defined

In [53]:
# Adjust the batch size to match your TensorFlow dataset
batch_size = 16

# Calculate steps per epoch and total training steps
steps_per_epoch = len(tokenized_train) // batch_size
total_training_steps = steps_per_epoch * epochs

# Create an optimizer using the transformers utility
from transformers import create_optimizer

optimizer, schedule = create_optimizer(
    init_lr=learning_rate, 
    num_train_steps=total_training_steps, 
    num_warmup_steps=0
)

# Compile the model with the optimizer and loss function
model.compile(optimizer=optimizer, loss=model.compute_loss)
model.summary()

# Assuming you've created `tf_valid_dataset` in a similar manner as `tf_train_dataset`
# Train the model
# model.fit(
#     tf_train_dataset,
#     validation_data=tf_valid_dataset,
#     epochs=epochs
# )

Model: "tfm_bart_for_conditional_generation_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFMBartMainLayer)    multiple                  420361216 
                                                                 
 final_logits_bias (BiasLay  multiple                  64001     
 er)                                                             
                                                                 
Total params: 420425217 (1.57 GB)
Trainable params: 420361216 (1.57 GB)
Non-trainable params: 64001 (250.00 KB)
_________________________________________________________________


In [26]:
def summarize_text(texts, max_length=150, min_length=40):
    """
    Generate summaries for a list of input texts using the trained model.
    
    Args:
    - texts: List of strings containing the texts to summarize.
    - max_length: Maximum length of the summary.
    - min_length: Minimum length of the summary.
    
    Returns:
    - List of generated summaries.
    """
    # Tokenize inputs
    inputs = tokenizer(texts, max_length=1024, truncation=True, padding="max_length", return_tensors="tf")
    
    # Generate summaries
    summaries = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        min_length=min_length,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    
    # Decode summaries
    return [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in summaries]

# Example usage:
input_texts = [
    "Giá vàng sẽ tiếp_tục tăng trong dài hạn, một phần do tình_hình tài_chính bấp_bênh của nhiều quốc_gia phương Tây"
]

summarized_texts = summarize_text(input_texts)
for i, summary in enumerate(summarized_texts):
    print(f"Summary {i+1}: {summary}")


Summary 1: Giá vàng sẽ tiếp_tục tăng trong dài hạn, một phần do tình_hình tài_chính bấp_bênh của nhiều quốc_gia phương Tây( * ) : 1 - 1 - 1 - 1 - 1 - 1 - 1


----

In [49]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from transformers import TFBertModel

class ScaledDotProductAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ScaledDotProductAttention, self).__init__(**kwargs)

    def call(self, query, key, value, mask=None):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        dk = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        if mask is not None:
            scaled_attention_logits += (mask * -1e9)

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0
        self.depth = d_model // self.num_heads

        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)

        self.dense = Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, _ = ScaledDotProductAttention()(q, k, v, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        output = self.dense(concat_attention)
        return output
class TransformerEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, x, training, mask):
        attn_output = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2


In [55]:
# Thay đổi phần gọi mô hình BERT trong hàm build_model
def build_model(vocab_size, max_len, d_model, num_heads, dff):
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

    bert_model = TFBertModel.from_pretrained("vinai/phobert-base")

    # Chuyển đổi Keras Tensors thành TensorFlow tensors
    input_ids_tensor = tf.convert_to_tensor(input_ids)
    attention_mask_tensor = tf.convert_to_tensor(attention_mask)

    bert_outputs = bert_model(input_ids_tensor, attention_mask=attention_mask_tensor)
    x = bert_outputs.last_hidden_state

    encoder_layer = TransformerEncoderLayer(d_model, num_heads, dff)
    x = encoder_layer(x, training=True, mask=None)
    x = Dense(vocab_size, activation='softmax')(x)

    model = Model(inputs=[input_ids, attention_mask], outputs=x)
    return model


In [56]:
model = build_model(vocab_size=30000, max_len=512, d_model=512, num_heads=8, dff=2048)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": tokenized_train["input_ids"], 
                                                     "attention_mask": tokenized_train["attention_mask"]}, 
                                                    tokenized_train["labels"])).batch(32)

model.fit(train_dataset, epochs=3)


You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFBertModel: ['lm_head', 'roberta']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
